# Mixture of Experts(MoE)

In [ ]:
#|default_exp moe

In [ ]:
#|export
import math, torch, matplotlib as mpl, matplotlib.pyplot as plt
from torch import nn
from fastai.vision.all import *

In [ ]:
#|hide
import math
import torch
import torchvision.transforms as T
import matplotlib as mpl
import torch.nn as nn, torch.nn.functional as F, torch.optim as optim
from fastai.vision.all import *
from torchvision.datasets import MNIST

In [ ]:
#|hide
set_seed(0)
n_cores = num_cpus()


bs = 256 # batch size
h = 256

In [ ]:
#|hide
mean, std = 0.130652368068695068, 0.307504087686538696

train_dataset = MNIST('../data', train=True, download=True,
                             transform=T.Compose([
                                 T.ToTensor(),
                                 T.Normalize((mean,), (std,)),T.Lambda(lambda x: x.view(-1,28*28).squeeze())]))
test_dataset = MNIST('../data', train=False, download=True,
                            transform=T.Compose([
                                T.ToTensor(),
                                T.Normalize((mean,), (std,)),T.Lambda(lambda x: x.view(-1,28*28).squeeze())]))
dls = DataLoaders(TfmdDL(train_dataset, bs, True, n_cores),TfmdDL(test_dataset, bs, False, n_cores))

In [ ]:
#|hide
class MoE(nn.Module):
    def __init__(self, in_len, out_len, n_exps=4, act=nn.ReLU):
        self.act = act()
        self.n = n_exps
        self.router = nn.Linear(in_len, self.n)
        self.experts = nn.Linear(in_len, self.n*out_len)